In [1]:
import glob
import pandas as pd
import string
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import math
import csv
import re
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV 
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectFromModel
import us

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/seanharding/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
## anything missing? Can install missing items 
## (like the package us and the latest version of scikit-learn) 
## like this:
import sys
!{sys.executable} -m pip install us
!{sys.executable} -m pip install --upgrade scikit-learn
#then 'Kernel:Restart'

In [2]:
congressSpeech = "data/covidUtterances.tsv"
outfile = "data/fullPredictions.tsv"
articleList = glob.glob("data/article*") #glob is a fuzzy matcher
print(articleList) #do I have everything?

## Judgment calls time: choosing stop words!
my_stop = [state.name.lower() for state in us.states.STATES] #list of all US states should be stop words; congressional uses of state names are collinear with the congressperson
my_stop = my_stop + ['democrat', 'democratic', 'republican', 'democrats', 'republicans']
my_stop = my_stop + ['leader mcconnell', 'speaker pelosi', 'representative', 'house', 'armed services', 'contact', 'remarks',  'prepared', 'delivery', 'opening', 'statement', 'mike', 'rogers', 'chairman', 'adam', 'smith', 'joe', 'wilson', 'joe', 'courtney', 'michael', 'turner', 'john', 'garamendi', 'doug', 'lamborn', 'donald', 'norcross', 'robert', 'wittman', 'ruben', 'gallego', 'Mike Rogers', 'Joe Wilson', 'Michael Turner', 'Doug Lamborn', 'Robert Wittman', 'Austin Scott', 'Sam Graves', 'Elise Stefanik', 'Scott DesJarlais', 'Trent Kelly', 'Mike Gallagher', 'Matt Gaetz', 'Don Bacon', 'Jim Banks', 'Jack Bergman', 'Michael Waltz',  'Mike Johnson', 'Lisa McClain', 'Ronny Jackson', 'Pat Fallon', 'Carlos Gimenez', 'Nancy Mace', 'Brad Finstad', 'Dale Strong', 'Morgan Luttrell', 'Jennifer Kiggans', 'Nick LaLota', 'James Moylan', 'Mark Alford', 'Cory Mills', 'Richard McCormick', 'Adam Smith', 'Joe Courtney', 'John Garamendi', 'Donald Norcross', 'Ruben Gallego', 'Seth Moulton', 'Salud Carbajal', 'Ro Khanna', 'William Keating', 'Andy Kim', 'Chrissy Houlahan', 'Elissa Slotkin', 'Mikie Sherrill', 'Veronica Escobar', 'Jared Golden', 'Sara Jacobs', 'Marilyn Strickland', 'Patrick Ryan', 'Jeff Jackson', 'Gabe Vasquez', 'Christopher Deluzio', 'Jill Tokuda', 'Donald Davis', 'Jennifer McClellan', 'Terri Sewell', 'Steven Horsford', 'Jimmy Panetta', 'Marc Veasey']
my_stop = my_stop + ['carolina', 'dakota', 'hampshire', 'island', 'jersey', 'mexico', 'new', 'north', 'rhode', 'south', 'west', 'york'] #these are spit out by the tokenizer
stop = my_stop + stopwords.words('english')



['data/articleWikipedia.txt', 'data/article1638.html']


In [3]:
dDF = pd.read_csv("data/democrat.tsv", sep='\t', header=0)
rDF = pd.read_csv("data/republican.tsv", sep='\t', header=0)
dDF['is_republican'] = 0
rDF['is_republican'] = 1
frames = [dDF, rDF]
inDF = pd.concat(frames)
#inDF.rename(columns={"Text": "speech"})

In [ ]:
#my_stop ## what's in the list now?

In [48]:
#inDF = pd.read_csv(congressSpeech, sep='\t', header=0)
inDF.head(3)

,URL,Title,Date,Text,is_republican
0,https://democrats-armedservices.house.gov/pres...,Smith Statement in Honor of Memorial Day,2023-05-29T04:00:00Z,"SEATTLE, WA– Today, Representative Adam Smith ...",0
1,https://democrats-armedservices.house.gov/pres...,Smith Statement on Nomination of Air Force Gen...,2023-05-25T04:00:00Z,"WASHINGTON, D.C.– Today, Representative Adam S...",0
2,https://democrats-armedservices.house.gov/pres...,Smith Statement on Postponement of FY24 NDAA M...,2023-05-10T04:00:00Z,"WASHINGTON, D.C. –Representative Adam Smith (D...",0


In [4]:

#cleanup routine
inDF['Text'] = inDF['Text'].apply(lambda x: str(x)) #stringify everything
numInIt = re.compile('\d')
inDF['Text']= inDF['Text'].apply(lambda x: ' '.join(x for x in x.split() if not numInIt.search(x))) #eliminate numbers and words that contain numbers
inDF['Text'] = inDF['Text'].apply(lambda x: x.lower()) #lowercase everything
inDF['Text'] = inDF['Text'].apply(lambda x: x.lstrip()) #eliminate leading whitespace
inDF['Text'] = inDF['Text'].apply(lambda x: ' '.join(x for x in x.split() if x not in string.punctuation)) #elim punctuation

inDF = inDF.dropna()
inDF = inDF.reset_index(drop=True)
minusNA = inDF.shape
inDF.head(3)

,URL,Title,Date,Text,is_republican
0,https://democrats-armedservices.house.gov/pres...,Smith Statement in Honor of Memorial Day,2023-05-29T04:00:00Z,"seattle, wa– today, representative adam smith ...",0
1,https://democrats-armedservices.house.gov/pres...,Smith Statement on Nomination of Air Force Gen...,2023-05-25T04:00:00Z,"washington, d.c.– today, representative adam s...",0
2,https://democrats-armedservices.house.gov/pres...,Smith Statement on Postponement of FY24 NDAA M...,2023-05-10T04:00:00Z,"washington, d.c. –representative adam smith (d...",0


In [5]:
## what predicts what?
y = inDF['is_republican']
X = inDF['Text']

## two tasks in the pipeline: tfidf and ridge regression
estimators = [("tf_idf", TfidfVectorizer(ngram_range=(1,3), stop_words=stop)), ("ridge", linear_model.Ridge())]
model = Pipeline(estimators)



In [6]:
model = model.fit(X, y) #this is all it takes!
#if we do any hyper parameter tuning it can happen in here
finalModel = model 

/Users/seanharding/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['alford', 'andy', 'armed', 'austin', 'bacon', 'banks', 'bergman', 'brad', 'carbajal', 'carlos', 'chrissy', 'christopher', 'cory', 'dale', 'davis', 'deluzio', 'desjarlais', 'elise', 'elissa', 'escobar', 'fallon', 'finstad', 'gabe', 'gaetz', 'gallagher', 'gimenez', 'golden', 'graves', 'horsford', 'houlahan', 'jack', 'jackson', 'jacobs', 'james', 'jared', 'jeff', 'jennifer', 'jill', 'jim', 'jimmy', 'johnson', 'keating', 'kelly', 'khanna', 'kiggans', 'kim', 'lalota', 'leader', 'lisa', 'luttrell', 'mace', 'marc', 'marilyn', 'mark', 'matt', 'mcclain', 'mcclellan', 'mcconnell', 'mccormick', 'mikie', 'mills', 'morgan', 'moulton', 'moylan', 'nancy', 'nick', 'panetta', 'pat', 'patrick', 'pelosi', 'richard', 'ro', 'ronny', 'ryan', 'salud', 'sam', 'sara', 'scott', 'services', 'seth', '

In [7]:

print(f"For the model I now have {finalModel}")
tf_idf_model = finalModel.named_steps["tf_idf"]
ridge_model = finalModel.named_steps["ridge"]
print(tf_idf_model)
print(ridge_model)
coefficients = pd.DataFrame({"names":tf_idf_model.get_feature_names_out(), "coef":ridge_model.coef_})
print("Best")
print(coefficients.sort_values("coef", ascending=False).head(50))
print("Worst")
print(coefficients.sort_values("coef", ascending=False).tail(50))

For the model I now have Pipeline(steps=[('tf_idf',
                 TfidfVectorizer(ngram_range=(1, 3),
                                 stop_words=['alabama', 'alaska', 'arizona',
                                             'arkansas', 'california',
                                             'colorado', 'connecticut',
                                             'delaware', 'florida', 'georgia',
                                             'hawaii', 'idaho', 'illinois',
                                             'indiana', 'iowa', 'kansas',
                                             'kentucky', 'louisiana', 'maine',
                                             'maryland', 'massachusetts',
                                             'michigan', 'minnesota',
                                             'mississippi', 'missouri',
                                             'montana', 'nebraska', 'nevada',
                                             'new hampshire', 'new jersey

In [23]:
## now we bring in the article text and clean it up, then predict

outFH = open(outfile, 'w') #rewrites outfile fresh
outFH.write("articleID\tprediction\n")

for article in articleList:
        with open(article) as f:
                s_article = " ".join([l.rstrip() for l in f])
                p = re.compile(r'<.*?>') #tag killing regex
                articleClean = re.sub(p, '', s_article) #drops the tags

##Same cleanup routine as above
                articleClean = articleClean.lower() #lowercase everything
                articleClean = articleClean.lstrip() #eliminate leading whitespace 
                prediction = finalModel.predict([articleClean])
                print(f"Article is {article} and prediction is {prediction}")
                justID = article.split('/')[1]
                justID = justID.split('.')[0]
                justID = justID.lstrip('article')
                outFH.write(f"{justID}\t{prediction[0]}\n")

Article is data/articleWikipedia.txt and prediction is [0.1845735]
Article is data/article1638.html and prediction is [0.38833107]
